In [2]:
import pickle
import azureml

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from string import punctuation
import re


In [3]:
# from azureml.core import Workspace
# from azureml.core.compute import ComputeTarget

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig
from azureml.interpret import ExplanationClient

In [4]:
ws = Workspace.get(name = 'Azure_DS_workspace',
                  subscription_id='7910d7f1-3a32-4f77-b718-0a9b8bb1db72',
                  resource_group='Azure_DS')

In [5]:
ws.get_details()

{'id': '/subscriptions/7910d7f1-3a32-4f77-b718-0a9b8bb1db72/resourceGroups/Azure_DS/providers/Microsoft.MachineLearningServices/workspaces/Azure_DS_workspace',
 'name': 'Azure_DS_workspace',
 'location': 'westus',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {},
 'sku': 'Basic',
 'workspaceid': 'da89adbc-419f-461f-83ff-ec7f695680d8',
 'sdkTelemetryAppInsightsKey': '23040a33-1c3e-4782-a462-da85bf9238bb',
 'description': '',
 'friendlyName': 'Azure_DS_workspace',
 'creationTime': '2021-05-01T18:08:33.9866295+00:00',
 'containerRegistry': '/subscriptions/7910d7f1-3a32-4f77-b718-0a9b8bb1db72/resourceGroups/Azure_DS/providers/Microsoft.ContainerRegistry/registries/da89adbc419f461f83ffec7f695680d8',
 'keyVault': '/subscriptions/7910d7f1-3a32-4f77-b718-0a9b8bb1db72/resourcegroups/azure_ds/providers/microsoft.keyvault/vaults/azuredsworkspa7950443874',
 'applicationInsights': '/subscriptions/7910d7f1-3a32-4f77-b718-0a9b8bb1db72/resourcegroups/azure_ds/providers/microsoft.in

In [11]:
pwd

'/home/jawangcode/DSI/CommonLit/James'

In [12]:
import pandas as pd

train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [13]:
# ------------------------------------
def word_lenth_counts(text):
    '''
        Return: 
            Dictionry of words length counts 
    '''
    
    d = {"1_letter": 0, 
         "2_letter": 0, 
         "3_letter": 0, 
         "4_letter": 0, 
         "5_letter": 0, 
         "6_letter": 0, 
         "7_letter": 0, 
         "8_letter": 0, 
         "9_letter": 0, 
         "10_letter": 0, 
         "11_letter": 0, 
         "12_letter": 0
    }

    text.replace('\n', '')
    text = text.lower()
    n_text = ''
    for c in text: 
        if c not in punctuation:
            n_text += c
        else: n_text += ' '

    text = n_text
    for word in text.split(' '): 
        wl = len(word)
        if wl > 0:
            if len(word) >= 12: key = f"12_letter"
            else:  key = f"{len(word)}_letter"

            d[key] += 1; 
    return d
# ------------------------------------
def source_info(text):
    '''
        Return: 
            Dictionary of source wiki, article, book, details, story or stories, kid, edu, simple
    '''
    d = dict()
    source_type = ['wiki', 'article', 'book', 'details', 'kid', 'edu', 'simple', 'story', 'stories']
    
    for t in source_type:
        if t != 'stories': 
            d[t] = 0
            
        if t in text:
            if t == 'stories': 
                d['story'] = 1;
            else: d[t] = 1
    return d
# ------------------------------------
def document_info(text):
    '''
        Return:
            Dictionary of document lenght, word count, sentence count, average word lenght
    '''
    text_lenght = len(text)
    text = text.replace('\n', ' ')
    text_word_count = len(text.split(' '))
    text_sentence_count = len(text.split('.'))
    text_avg_word_length = round(sum([len(t) for t in text.split(' ')]) / text_word_count, 2)

    document_info = {
        'doc_len': text_lenght,
        'word_count': text_word_count,
        'sent_count': text_sentence_count,
        'avg_word_len': text_avg_word_length
    }

    return document_info
    
# ------------------------------------
def character_counts(text):
    '''
        Return:
            Dictionary of counts of all characters in text
    '''
    char_dict = dict()
    text = text.lower()
    text = text.replace('\n', ' ')
    
    for char in text:
        if char not in char_dict:
            char_dict[char] = 0
        
        char_dict[char] += 1     
        
    return char_dict

# ------------------------------------
def phonemes_counts(text):
    '''
        Return: 
            Dictionay of all phonemic in text
    '''
    phonemes = ['ck', 'cc', 'di', 'nn', 'dd', 'ai', 'ss', 'mn', 'bb', 
                'sci', 'ze', 'qu', 'se', 'sc', 'ci', 'ps', 'si', 'tch', 
                'ngue', 'st', 'gu', 'th', 'pn', 've', 'te', 'zz', 'au', 
                'lm', 'lf', 'ge', 'wh', 'tu', 'wr', 'ph', 'sh', 'mm', 'gh', 
                'dge', 'ft', 'tt', 'ed', 'ng', 'lk', 'ti', 'gue', 'rr', 'ch', 
                'll', 'gn', 'ff', 'gg', 'pp', 'rh', 'ce', 'mb', 'kn', 
                'eer', 'ere', 'uy', 'ho', 'ear', 'ei', 'ar', 'ai', 
                'oor', 'ure', 'eigh', 'ey', 'is', 'ae', 'ow', 'or', 'ew', 
                'ore', 'ur', 'uoy', 'air', 'au', 'ough', 'yr', 
                'ea', 'ayer', 'augh', 'aw', 'eau', 'aigh', 'igh', 'oy', 
                'oo', 'ue', 'are', 'ee', 'oa', 'et', 'y', 'er', 'eir', 
                'oew', 'oar', 'ie', 'eo', 'ui', 'ier', 'ou', 'ir', 'oi', 
                'ay', 'ye', 'oe', 'our']
    temp_dict = dict()
    
    # lower text 
    text = text.lower()
    
    for p in phonemes:
        temp_dict[p] = text.count(p)
    
    return temp_dict

In [14]:
#combine train and test set for pre-processing

c_df = pd.concat([train, test], axis = 0)
c_df.shape

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



(2841, 6)

In [15]:
# series_documents =  train.excerpt.apply(lambda x: document_info(x))
# series_phonemes = train.excerpt.apply(lambda x: phonemes_counts(x))
# series_characters =  train.excerpt.apply(lambda x: character_counts(x))
# series_word_length = train.excerpt.apply(lambda x: word_lenth_counts(x))

# series_documents_test =  test.excerpt.apply(lambda x: document_info(x))
# series_phonemes_test = test.excerpt.apply(lambda x: phonemes_counts(x))
# series_characters_test =  test.excerpt.apply(lambda x: character_counts(x))
# series_word_length_test = test.excerpt.apply(lambda x: word_lenth_counts(x))

series_documents =  c_df.excerpt.apply(lambda x: document_info(x))
series_phonemes = c_df.excerpt.apply(lambda x: phonemes_counts(x))
series_characters =  c_df.excerpt.apply(lambda x: character_counts(x))
series_word_length = c_df.excerpt.apply(lambda x: word_lenth_counts(x))

In [16]:
df_documents = pd.DataFrame(list(series_documents))
df_phonemes = pd.DataFrame(list(series_phonemes))
df_chracters = pd.DataFrame(list(series_characters))
df_word_lenght = pd.DataFrame(list(series_word_length))

# df_documents_test = pd.DataFrame(list(series_documents_test))
# df_phonemes_test = pd.DataFrame(list(series_phonemes_test))
# df_chracters_test = pd.DataFrame(list(series_characters_test))
# df_word_lenght_test = pd.DataFrame(list(series_word_length_test))


In [17]:
# Make na 0
df_chracters.fillna(0, inplace = True)
#df_chracters_test.fillna(0, inplace = True)


# Make all ints (change NaN to 0)
for col in df_chracters.columns:
    df_chracters[col] = df_chracters[col].astype(int, copy=False)

#for col in df_chracters_test.columns:
#    df_chracters_test[col] = df_chracters_test[col].astype(int, copy=False)

    

In [18]:
df_y = np.round(train['target'], 2)

In [19]:
df_X = pd.concat([df_documents, df_phonemes, df_word_lenght,df_y], axis = 1)
#df_X_test = pd.concat([df_documents_test, df_phonemes_test, df_word_lenght_test], axis = 1)


In [20]:
df_X = df_X.T.drop_duplicates().T
# df_X_test = df_X_test.T.drop_duplicates().T


In [21]:
df_X_test = df_X.iloc[2834:]
df_X = df_X.iloc[:2834]
df_X

,doc_len,word_count,sent_count,avg_word_len,ck,cc,di,nn,dd,ai,...,4_letter,5_letter,6_letter,7_letter,8_letter,9_letter,10_letter,11_letter,12_letter,target
0,992.0,179.0,12.0,4.55,1.0,0.0,1.0,0.0,0.0,1.0,...,49.0,21.0,10.0,9.0,10.0,8.0,3.0,0.0,0.0,-0.34
1,937.0,169.0,11.0,4.55,0.0,0.0,1.0,1.0,1.0,3.0,...,30.0,17.0,15.0,12.0,5.0,2.0,5.0,2.0,0.0,-0.32
2,908.0,166.0,12.0,4.48,3.0,1.0,1.0,1.0,1.0,4.0,...,35.0,24.0,7.0,11.0,9.0,3.0,2.0,0.0,0.0,-0.58
3,909.0,164.0,6.0,4.55,0.0,0.0,0.0,0.0,0.0,3.0,...,31.0,21.0,20.0,15.0,4.0,1.0,1.0,0.0,1.0,-1.05
4,723.0,147.0,6.0,3.93,0.0,0.0,0.0,0.0,7.0,4.0,...,28.0,28.0,14.0,0.0,2.0,0.0,0.0,0.0,0.0,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2829,901.0,145.0,11.0,5.22,1.0,0.0,12.0,2.0,0.0,1.0,...,27.0,32.0,10.0,12.0,4.0,9.0,7.0,0.0,2.0,1.71
2830,890.0,164.0,14.0,4.43,4.0,0.0,1.0,0.0,0.0,1.0,...,37.0,24.0,18.0,7.0,5.0,7.0,1.0,0.0,1.0,0.19
2831,949.0,190.0,17.0,4.00,0.0,0.0,1.0,0.0,1.0,1.0,...,36.0,18.0,20.0,15.0,2.0,3.0,0.0,0.0,1.0,0.26
2832,864.0,150.0,13.0,4.77,0.0,0.0,1.0,0.0,0.0,3.0,...,28.0,23.0,19.0,5.0,7.0,4.0,3.0,0.0,1.0,-0.22


In [22]:
df_X_test

,doc_len,word_count,sent_count,avg_word_len,ck,cc,di,nn,dd,ai,...,4_letter,5_letter,6_letter,7_letter,8_letter,9_letter,10_letter,11_letter,12_letter,target
2834,772.0,149.0,9.0,4.19,3.0,2.0,2.0,0.0,1.0,6.0,...,21.0,21.0,14.0,18.0,1.0,5.0,1.0,0.0,0.0,NaN
2835,967.0,181.0,13.0,4.35,1.0,0.0,0.0,0.0,0.0,5.0,...,51.0,36.0,15.0,5.0,4.0,3.0,2.0,0.0,0.0,NaN
2836,948.0,174.0,14.0,4.45,0.0,0.0,2.0,1.0,0.0,5.0,...,34.0,14.0,18.0,11.0,11.0,3.0,1.0,0.0,0.0,NaN
2837,1144.0,182.0,8.0,5.29,0.0,1.0,13.0,0.0,0.0,0.0,...,24.0,19.0,11.0,18.0,23.0,6.0,9.0,7.0,1.0,NaN
2838,1094.0,168.0,7.0,5.52,2.0,0.0,5.0,0.0,0.0,0.0,...,18.0,18.0,11.0,26.0,8.0,18.0,6.0,6.0,0.0,NaN
2839,823.0,148.0,8.0,4.57,0.0,0.0,2.0,0.0,0.0,1.0,...,39.0,7.0,12.0,17.0,3.0,2.0,0.0,3.0,3.0,NaN
2840,894.0,161.0,16.0,4.56,1.0,0.0,0.0,0.0,0.0,0.0,...,22.0,20.0,16.0,12.0,4.0,9.0,3.0,0.0,1.0,NaN


In [23]:
from sklearn.model_selection import train_test_split

x_train, x_test = train_test_split(df_X, test_size=0.2, random_state=223)

In [24]:
import logging

automl_settings = {
    "iteration_timeout_minutes": 10,
    "experiment_timeout_hours": 0.3,
    "enable_early_stopping": True,
    "primary_metric": 'normalized_root_mean_squared_error',
    "featurization": 'auto',
    "verbosity": logging.INFO,
    "n_cross_validations": 5
}

In [25]:
automl_config = AutoMLConfig(task = 'regression',
                             debug_log = 'automl_errors.log',
                             training_data = x_train,
                             label_column_name = 'target',
                             **automl_settings
                            )

In [26]:
experiment_name = 'Commonlit-local-Marketne-changes'
experiment=Experiment(ws, experiment_name)

output = {}
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T


,
Subscription ID,7910d7f1-3a32-4f77-b718-0a9b8bb1db72
Workspace,Azure_DS_workspace
Resource Group,Azure_DS
Location,westus
Experiment Name,Commonlit-local-Marketne-changes


In [27]:
local_run = experiment.submit(automl_config, show_output = True)

Running on local machine
Parent Run ID: AutoML_5819d4fc-bf4b-4578-b9cb-f5e2263cb890

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardi

WARNING - Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', OSError("(104, 'ECONNRESET')"))': /azureml/ExperimentRun/dcid.AutoML_5819d4fc-bf4b-4578-b9cb-f5e2263cb890_19/outputs/conda_env_v_1_0_0.yml?sv=2019-02-02&sr=b&sig=Qh4pds5d0Ohcb7ZMGUJKXmzB9Xiw1%2F3fN%2FF8udXblzo%3D&st=2021-05-21T20%3A11%3A13Z&se=2021-05-22T20%3A21%3A13Z&sp=rcw&comp=block&blockid=TURBd01EQXdNREF3TURBd01EQXdNREF3TURBd01EQXdNREF3TURBd01EQSUzRA%3D%3D&timeout=30


MinMaxScaler DecisionTree                      0:00:15       0.1712    0.1478
        20   StandardScalerWrapper LassoLars                0:00:16       0.1488    0.1478
        21   StandardScalerWrapper DecisionTree             0:00:15       0.1727    0.1478
        22   RobustScaler RandomForest                      0:00:18       0.1581    0.1478
        23   MaxAbsScaler RandomForest                      0:00:18       0.1654    0.1478
        24   MaxAbsScaler GradientBoosting                  0:00:37       0.1496    0.1478
        25   PCA ElasticNet                                 0:00:15       0.1917    0.1478
        26   RobustScaler RandomForest                      0:00:15       0.1592    0.1478
        27   

WARNING - Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', OSError("(104, 'ECONNRESET')"))': /azureml/ExperimentRun/dcid.AutoML_5819d4fc-bf4b-4578-b9cb-f5e2263cb890_27/outputs/model.pkl?sv=2019-02-02&sr=b&sig=57u9QQdvZdtoeyhD04yXiCeQ7vHS%2BPP3fcy0BTZlyS4%3D&st=2021-05-21T20%3A14%3A19Z&se=2021-05-22T20%3A24%3A19Z&sp=rcw&comp=block&blockid=TURBd01EQXdNREF3TURBd01EQXdNREF3TURBd01EQXdNREF3TURBd01EQSUzRA%3D%3D&timeout=30
WARNING - Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', OSError("(104, 'ECONNRESET')"))': /azureml/ExperimentRun/dcid.AutoML_5819d4fc-bf4b-4578-b9cb-f5e2263cb890_27/outputs/scoring_file_v_1_0_0.py?sv=2019-02-02&sr=b&sig=vSjiYR6lDwdBD%2B8TaAycu5SHbc45ZSLZQ3nHvp46iks%3D&st=2021-05-21T20%3A14%3A19Z&se=2021-05-22T20%3A24%3A19Z&sp=rcw&comp=block&blockid=TURBd01EQXdNREF3TURBd01EQXdNREF3TURB

StandardScalerWrapper ExtremeRandomTrees       0:00:35       0.1544    0.1478
        28   MinMaxScaler RandomForest                      0:00:17       0.1608    0.1478
        29   StandardScalerWrapper RandomForest             0:01:12       0.1497    0.1478
        30   VotingEnsemble                                 0:00:55       0.1446    0.1446
        31   StackEnsemble                                  0:00:52       0.1447    0.1446
Stopping criteria reached at iteration 32. Ending experiment.
****************************************************************************************************
Current status: BestRunExplainModel. Best run model explanations started
Current status: ModelExplanationDataSetSetup. Model explanations data setup completed
Current status: PickSurrogateModel. Choosing LightGBM as the surrogate model for explanations
Current status: EngineeredFeatureExplanations. Computation of engineered features started
Current status: EngineeredFeatureExplanations. Compu

In [1]:
best_run, fitted_model = local_run.get_output()
print(best_run)
print(fitted_model)

NameError: name 'local_run' is not defined

In [129]:
y_test = x_test.pop('target')

y_predict = fitted_model.predict(x_test)
print(y_predict[:10])

[ 0.32581944 -0.11617363 -1.12854974 -1.18239724 -1.27583984 -0.29739247
 -1.94680389 -1.99770246 -1.0185855  -1.53932385]


In [130]:
from sklearn.metrics import mean_squared_error
from math import sqrt

y_actual = y_test.values.flatten().tolist()
rmse = sqrt(mean_squared_error(y_actual, y_predict))
rmse

0.747579762355829

## Submission

In [134]:
df_X_test = df_X_test.drop(['target'],axis=1)

y_predict = fitted_model.predict(df_X_test)
print(y_predict[:10])

[-0.69944621 -0.16837356 -0.58493931 -2.16809658 -1.2875082  -1.1111762
 -0.09716158]


In [ ]:
submission = pd.read_csv()

In [ ]:
score()

RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=None,
                                                             enable_feature_sweeping=None,
                                                             feature_sweeping_config=None,
                                                             feature_sweeping_timeout=None,
                                                             featurization_config=None,
                                                             force_text_dnn=None,
                                                             is_cross_validation=None,
                                                             is_onnx_compatible=None,
                                                             logger=None,
                                                             observer=None,
                                         